In [1]:
import requests
import pandas as pd
import numpy as np
import random

from geopy.geocoders import Nominatim

#libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

import folium

print('All libraries imported.')

All libraries imported.


##### To obtain access token follow these steps.

<br>

1.  Go to your **"App Settings"** page on the developer console of Foursquare.com   
2.  Set the **"Redirect URL"** under **"Web Addresses"** to [https://www.google.com](https://www.google.com?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)  


3.  Paste and enter the following url in your web browser **(replace YOUR_CLIENT_ID with your actual client id)**:  
    [https://foursquare.com/oauth2/authenticate?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=https://www.google.com](https://foursquare.com/oauth2/authenticate?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=https://www.google.com&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) 

    This should redirect you to a google page requesting permission to make the connection. 
4.  Accept and then look at the url of your web browser **(take note at the CODE part of the url to use in step 5)**  
    It should look like [https://www.google.com/?code=CODE](https://www.google.com?code=CODE&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)  
5.  Copy the code value from the previous step.  
       Paste and enter the following into your web browser **(replace placeholders with actual values)**:  
    [https://foursquare.com/oauth2/access_token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_CLIENT_SECRET&grant_type=authorization_code&redirect_uri=https://www.google.com&code=CODE](https://foursquare.com/oauth2/access_token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_CLIENT_SECRET&grant_type=authorization_code&redirect_uri=https://www.google.com&code=CODE&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).  
6.  When you paste the link , This should lead you to a page that gives you your **access token**.


In [2]:
# https://www.google.com/?code=IWAOTG2TFCUJJG5ZSQH4TL0HLNE3ZTDC4LYC5M4QETUZQZZC#_=_
# access_token: MQYF24WMEA5Z311K0ZAQVXYR2FSDYGMBQO2LIO52TFSRPIHR

In [3]:
CLIENT_ID = 'YWVH0GGCLFUBSXW54AVU5QZGBC04B2YPSGMAEHEUG5KFLXJL'
CLIENT_SECRET = 'KXBHLTKQDXOKCTYYGU5KUTAD1QWVJAZHICXBFBTWE5YOEZGF'
ACCESS_TOKEN = 'MQYF24WMEA5Z311K0ZAQVXYR2FSDYGMBQO2LIO52TFSRPIHR'
VERSION = '20211021'
LIMIT = 30
print('Your credentials are: Client ID is {}, Client Secret is {}.'.format(CLIENT_ID, CLIENT_SECRET))

Your credentials are: Client ID is YWVH0GGCLFUBSXW54AVU5QZGBC04B2YPSGMAEHEUG5KFLXJL, Client Secret is KXBHLTKQDXOKCTYYGU5KUTAD1QWVJAZHICXBFBTWE5YOEZGF.


In [42]:
address = 'Geschwister-Scholl-Platz 1, Munich, Germany'

geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
LATITUDE = location.latitude
LONGITUDE = location.longitude
print(LATITUDE, LONGITUDE)

48.15047575 11.579835648337658


### 1. Searching venues around a specified location

In [6]:
QUERY = 'Italian'
RADIUS = 1000
url = 'https://api.foursquare.com/v2/venues/search?cleint_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, LATITUDE, LONGITUDE, ACCESS_TOKEN, VERSION, QUERY, RADIUS, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']

df = pd.json_normalize(venues)
df

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,5852ed52ac3291412330aa68,The Italian Shot,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1635021234,False,Theresienstr. 40,48.148114,11.575081,"[{'label': 'display', 'lat': 48.14811439351411...",440,80333,DE,Universität,München,Bayern,Deutschland,"[Theresienstr. 40, 80333 München]",NaN,NaN
1,4ade0ceaf964a520c26921e3,Osteria Italiana,"[{'id': '55a5a1ebe4b013909087cb7f', 'name': 'T...",v-1635021234,False,Schellingstr. 62,48.151427,11.572119,"[{'label': 'display', 'lat': 48.15142738620246...",582,80799,DE,NaN,München,Bayern,Deutschland,"[Schellingstr. 62 (Schraudolphstr.), 80799 Mün...",Schraudolphstr.,NaN
2,5a71c8e41f8ed61fd38b6341,Gelataria Italiana Et Piccola cucina,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1635021234,False,Turkenstrasse 59,48.151075,11.576495,"[{'label': 'display', 'lat': 48.151075, 'lng':...",256,80779,DE,NaN,München,Bayern,Deutschland,"[Turkenstrasse 59, 80779 München]",NaN,NaN
3,5022a44f64a4363fdfeffada,Antica Rosticceria Italiano,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1635021234,False,Kaulbachstrasse 36,48.150304,11.584171,"[{'label': 'display', 'lat': 48.15030434135035...",322,80539,DE,NaN,München,Bayern,Deutschland,"[Kaulbachstrasse 36 (Veterinärstrasse), 80539 ...",Veterinärstrasse,NaN
4,4e316689152071f36460c638,STUDIO ITALIANO,"[{'id': '52e81612bcbc57f1066b7a48', 'name': 'L...",v-1635021234,False,Franz-Joseph-Strasse 48,48.157830,11.575301,"[{'label': 'display', 'lat': 48.15783, 'lng': ...",885,80801,DE,NaN,München,Bayern,Deutschland,"[Franz-Joseph-Strasse 48, 80801 München]",NaN,NaN
5,56fff4bc498ef25189815d16,Cupido ''Arte di Cucina Italiana'',"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1635021234,False,Bruderstr. 8,48.142655,11.586719,"[{'label': 'display', 'lat': 48.142655, 'lng':...",1009,80538,DE,NaN,München,Bayern,Deutschland,"[Bruderstr. 8, 80538 München]",NaN,173980972
6,5721ea32498ee46a93b93cd1,Gusto Italiano Da Giovanna,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1635021234,False,NaN,48.140915,11.576415,"[{'label': 'display', 'lat': 48.140915, 'lng':...",1094,NaN,DE,NaN,München,Bayern,Deutschland,[München],NaN,NaN


In [7]:
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]

df_filtered

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,id
0,The Italian Shot,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",Theresienstr. 40,48.148114,11.575081,"[{'label': 'display', 'lat': 48.14811439351411...",440,80333,DE,Universität,München,Bayern,Deutschland,"[Theresienstr. 40, 80333 München]",NaN,5852ed52ac3291412330aa68
1,Osteria Italiana,"[{'id': '55a5a1ebe4b013909087cb7f', 'name': 'T...",Schellingstr. 62,48.151427,11.572119,"[{'label': 'display', 'lat': 48.15142738620246...",582,80799,DE,NaN,München,Bayern,Deutschland,"[Schellingstr. 62 (Schraudolphstr.), 80799 Mün...",Schraudolphstr.,4ade0ceaf964a520c26921e3
2,Gelataria Italiana Et Piccola cucina,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",Turkenstrasse 59,48.151075,11.576495,"[{'label': 'display', 'lat': 48.151075, 'lng':...",256,80779,DE,NaN,München,Bayern,Deutschland,"[Turkenstrasse 59, 80779 München]",NaN,5a71c8e41f8ed61fd38b6341
3,Antica Rosticceria Italiano,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",Kaulbachstrasse 36,48.150304,11.584171,"[{'label': 'display', 'lat': 48.15030434135035...",322,80539,DE,NaN,München,Bayern,Deutschland,"[Kaulbachstrasse 36 (Veterinärstrasse), 80539 ...",Veterinärstrasse,5022a44f64a4363fdfeffada
4,STUDIO ITALIANO,"[{'id': '52e81612bcbc57f1066b7a48', 'name': 'L...",Franz-Joseph-Strasse 48,48.157830,11.575301,"[{'label': 'display', 'lat': 48.15783, 'lng': ...",885,80801,DE,NaN,München,Bayern,Deutschland,"[Franz-Joseph-Strasse 48, 80801 München]",NaN,4e316689152071f36460c638
5,Cupido ''Arte di Cucina Italiana'',"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",Bruderstr. 8,48.142655,11.586719,"[{'label': 'display', 'lat': 48.142655, 'lng':...",1009,80538,DE,NaN,München,Bayern,Deutschland,"[Bruderstr. 8, 80538 München]",NaN,56fff4bc498ef25189815d16
6,Gusto Italiano Da Giovanna,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",NaN,48.140915,11.576415,"[{'label': 'display', 'lat': 48.140915, 'lng':...",1094,NaN,DE,NaN,München,Bayern,Deutschland,[München],NaN,5721ea32498ee46a93b93cd1


In [8]:
df.loc[0, 'categories']

[{'id': '4bf58dd8d48988d110941735',
  'name': 'Italian Restaurant',
  'pluralName': 'Italian Restaurants',
  'shortName': 'Italian',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
   'suffix': '.png'},
  'primary': True}]

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

df_filtered['categories'] = df_filtered.apply(get_category_type, axis = 1)
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

df_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,The Italian Shot,Italian Restaurant,Theresienstr. 40,48.148114,11.575081,"[{'label': 'display', 'lat': 48.14811439351411...",440,80333,DE,Universität,München,Bayern,Deutschland,"[Theresienstr. 40, 80333 München]",NaN,5852ed52ac3291412330aa68
1,Osteria Italiana,Trattoria/Osteria,Schellingstr. 62,48.151427,11.572119,"[{'label': 'display', 'lat': 48.15142738620246...",582,80799,DE,NaN,München,Bayern,Deutschland,"[Schellingstr. 62 (Schraudolphstr.), 80799 Mün...",Schraudolphstr.,4ade0ceaf964a520c26921e3
2,Gelataria Italiana Et Piccola cucina,Restaurant,Turkenstrasse 59,48.151075,11.576495,"[{'label': 'display', 'lat': 48.151075, 'lng':...",256,80779,DE,NaN,München,Bayern,Deutschland,"[Turkenstrasse 59, 80779 München]",NaN,5a71c8e41f8ed61fd38b6341
3,Antica Rosticceria Italiano,Italian Restaurant,Kaulbachstrasse 36,48.150304,11.584171,"[{'label': 'display', 'lat': 48.15030434135035...",322,80539,DE,NaN,München,Bayern,Deutschland,"[Kaulbachstrasse 36 (Veterinärstrasse), 80539 ...",Veterinärstrasse,5022a44f64a4363fdfeffada
4,STUDIO ITALIANO,Language School,Franz-Joseph-Strasse 48,48.157830,11.575301,"[{'label': 'display', 'lat': 48.15783, 'lng': ...",885,80801,DE,NaN,München,Bayern,Deutschland,"[Franz-Joseph-Strasse 48, 80801 München]",NaN,4e316689152071f36460c638
5,Cupido ''Arte di Cucina Italiana'',Italian Restaurant,Bruderstr. 8,48.142655,11.586719,"[{'label': 'display', 'lat': 48.142655, 'lng':...",1009,80538,DE,NaN,München,Bayern,Deutschland,"[Bruderstr. 8, 80538 München]",NaN,56fff4bc498ef25189815d16
6,Gusto Italiano Da Giovanna,Grocery Store,NaN,48.140915,11.576415,"[{'label': 'display', 'lat': 48.140915, 'lng':...",1094,NaN,DE,NaN,München,Bayern,Deutschland,[München],NaN,5721ea32498ee46a93b93cd1


In [10]:
df_filtered.name

0                        The Italian Shot
1                        Osteria Italiana
2    Gelataria Italiana Et Piccola cucina
3             Antica Rosticceria Italiano
4                         STUDIO ITALIANO
5      Cupido ''Arte di Cucina Italiana''
6              Gusto Italiano Da Giovanna
Name: name, dtype: object

In [43]:
venues_map = folium.Map([LATITUDE, LONGITUDE], zoom_start = 15)

folium.CircleMarker([LATITUDE, LONGITUDE],
                   radius = 20,
                   color = 'seagreen', border = 6,
                   popup = 'LMU München',
                   fill = True,
                   fill_color = 'white',
                   fill_opacity = 0.5).add_to(venues_map)

for lat, lon, name in zip(df_filtered.lat, df_filtered.lng, df_filtered.name):
    folium.Marker([lat, lon],
                 popup = name,
                 icon = folium.Icon(color = 'green')).add_to(venues_map)

venues_map

### 2. Exploring a specified venue

In [12]:
venue_id = '4ade0ceaf964a520c26921e3'
url_venue = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION)

In [14]:
result_venue = requests.get(url_venue).json()
result_venue

{'meta': {'code': 200, 'requestId': '617472884ddae851cd65743b'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venue': {'id': '4ade0ceaf964a520c26921e3',
   'name': 'Osteria Italiana',
   'contact': {'phone': '0892720717', 'formattedPhone': '089 2720717'},
   'location': {'address': 'Schellingstr. 62',
    'crossStreet': 'Schraudolphstr.',
    'lat': 48.151427386202464,
    'lng': 11.572119028409002,
    'labeledLatLngs': [{'label': 'display',
      'lat': 48.151427386202464,
      'lng': 11.572119028409002}],
    'postalCode': '80799',
    'cc': 'DE',
    'city': 'München',
    'state': 'Bayern',
    'country': 'Deutschland',
    'formattedAddress': ['Schellingstr. 62 (Schraudolphstr.)',
     '80799 München']},
   'canonicalUrl': 'https://foursquare.com/v/osteria-italiana/4ade0ceaf964a520c26921e3',
   'categories': [{'id': '55a5a1ebe4b013909087cb7f',
     'name': 'Trattoria/Osteria',
     'pluralName': 'Trattorias/Osterias',
     'shortNa

In [16]:
print(result_venue['response']['venue'].keys())

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


In [18]:
result_venue['response']['venue']

{'id': '4ade0ceaf964a520c26921e3',
 'name': 'Osteria Italiana',
 'contact': {'phone': '0892720717', 'formattedPhone': '089 2720717'},
 'location': {'address': 'Schellingstr. 62',
  'crossStreet': 'Schraudolphstr.',
  'lat': 48.151427386202464,
  'lng': 11.572119028409002,
  'labeledLatLngs': [{'label': 'display',
    'lat': 48.151427386202464,
    'lng': 11.572119028409002}],
  'postalCode': '80799',
  'cc': 'DE',
  'city': 'München',
  'state': 'Bayern',
  'country': 'Deutschland',
  'formattedAddress': ['Schellingstr. 62 (Schraudolphstr.)', '80799 München']},
 'canonicalUrl': 'https://foursquare.com/v/osteria-italiana/4ade0ceaf964a520c26921e3',
 'categories': [{'id': '55a5a1ebe4b013909087cb7f',
   'name': 'Trattoria/Osteria',
   'pluralName': 'Trattorias/Osterias',
   'shortName': 'Trattoria/Osteria',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 17},
 'url': 'http:

In [19]:
try: 
    print(result_venue['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


In [20]:
venue_id2 = '5852ed52ac3291412330aa68'
url_venue2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id2, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION)

In [22]:
result_venue2 = requests.get(url_venue2).json()

try:
    print(result_venue2['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.6


In [23]:
result_venue2['response']['venue']['tips']['count']

4

In [35]:
limit = 5
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&cleint_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION, limit)

result_tip = requests.get(url).json()
result_tip

{'meta': {'code': 200, 'requestId': '617476cfc8e81d5de9adaaf3'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 12,
   'items': [{'id': '51581ae7e4b0c7c76a2b88e1',
     'createdAt': 1364728551,
     'text': "Apparently predecessor Osteria Bavaria was Hitler's favourite restaurant in Munich. Largely unchanged besides the style of food since then one must try the ravioli and enjoy authentic Italian service.",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/51581ae7e4b0c7c76a2b88e1',
     'likes': {'count': 3,
      'groups': [{'type': 'others',
        'count': 3,
        'items': [{'id': '48068334',
          'firstName': 'Pung Worathiti',
          'lastName': 'Manosroi',
          'gender': 'male',
          'countryCode': 'DE',
          'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
           'suffix': '/48068334_YTz5UCo0_LtzpmomWFaxL4wfP6SW8gTLWuDm1ozC1AV-AKHVu5BRt4mVhFsEjCv-QmNyxO7HO.j

In [36]:
tips = result_tip['response']['tips']['items'][0]
tips.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [41]:
pd.set_option('display.max_colwidth', None)
tips_df = pd.json_normalize(tips)

filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.id
0,Apparently predecessor Osteria Bavaria was Hitler's favourite restaurant in Munich. Largely unchanged besides the style of food since then one must try the ravioli and enjoy authentic Italian service.,6,0,51581ae7e4b0c7c76a2b88e1,samichlaus,41467630


### 3. Exploring a specified user

In [44]:
user_id = '41467630'
url_user = 'https://api.foursquare.com/v2/users/{}/?client_id={}&client_secret={}&oauth_token={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION)

result_user = requests.get(url_user).json()
result_user

{'meta': {'code': 200, 'requestId': '6175233d3aa6db2e30103efd'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'user': {'id': '41467630',
   'firstName': 'samichlaus',
   'gender': 'male',
   'countryCode': 'AU',
   'canonicalUrl': 'https://foursquare.com/user/41467630',
   'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
    'suffix': '/GHN5RJ3O24Y5N0UK.jpg'},
   'friends': {'count': 23,
    'groups': [{'type': 'friends',
      'name': 'Mutual friends',
      'count': 0,
      'items': []},
     {'type': 'others',
      'name': 'Other friends',
      'count': 23,
      'items': [{'id': '10568963',
        'firstName': 'CECI',
        'lastName': 'GOMEZ MILLER',
        'gender': 'female',
        'countryCode': 'MX',
        'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
         'suffix': '/YOXTHDJC220UMCU3.jpg'},
        'tips': {'count': 1},
        'lists': {'groups': [{'type': 'created', 'count': 2, 'items': []}]},
 

In [46]:
user_data = result_user['response']['user']['photos']['items']
user_data

[{'id': '5e086bf06abf790008ff2a6d',
  'createdAt': 1577610224,
  'source': {'name': 'Swarm for iOS', 'url': 'https://www.swarmapp.com'},
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'suffix': '/41467630_1GTbZJncRsVjx9rvBpHm7YabGeKtHDKr-w2iI4TZMO0.jpg',
  'width': 1440,
  'height': 1920,
  'visibility': 'public',
  'venue': {'id': '56bc4b58498e4e11137e5eaa',
   'name': 'The Lawn Canggu',
   'location': {'address': 'Jl. Pura Dalem, Pantai Batu Bolong, Canggu',
    'lat': -8.603358875197229,
    'lng': 115.19734839701078,
    'labeledLatLngs': [{'label': 'display',
      'lat': -8.603358875197229,
      'lng': 115.19734839701078}],
    'postalCode': '80361',
    'cc': 'ID',
    'city': 'Badung',
    'state': 'Bali',
    'country': 'Indonesia',
    'formattedAddress': ['Jl. Pura Dalem, Pantai Batu Bolong, Canggu',
     'Badung',
     'Bali 80361',
     'Indonesia']},
   'categories': [{'id': '4bf58dd8d48988d128941735',
     'name': 'Cafeteria',
     'pluralName': 'Cafeterias',
   

In [47]:
pd.set_option('display.max_colwidth', None)

user_df = pd.json_normalize(user_data)
filtered_columns = ['id', 'prefix', 'suffix', 'width', 'height']
user_df_filtered = user_df.loc[:, filtered_columns]
user_df_filtered

,id,prefix,suffix,width,height
0,5e086bf06abf790008ff2a6d,https://fastly.4sqi.net/img/general/,/41467630_1GTbZJncRsVjx9rvBpHm7YabGeKtHDKr-w2iI4TZMO0.jpg,1440,1920


In [49]:
g = user_df.loc[tips_df['user.id'] == '41467630']
print('firstname: '+tips_df['user.firstName'])

0    firstname: samichlaus
Name: user.firstName, dtype: object


In [51]:
prefix = user_df_filtered.loc[0, 'prefix']
suffix = user_df_filtered.loc[0, 'suffix']
Image(url = '{}1440x1920{}'.format(prefix, suffix))

In [52]:
user_id

'41467630'

In [53]:
limit

5

In [54]:
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION, LIMIT)

In [66]:
results_usertip = requests.get(url).json()
usertips = results_usertip['response']['tips']['items']

pd.set_option('display.max_colwidth', None)

df_usertips = pd.json_normalize(usertips)
df_usertips.columns

Index(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photourl', 'like',
       'logView', 'agreeCount', 'disagreeCount', 'authorInteractionType',
       'photo.id', 'photo.createdAt', 'photo.source.name', 'photo.source.url',
       'photo.prefix', 'photo.suffix', 'photo.width', 'photo.height',
       'photo.visibility', 'likes.count', 'likes.groups', 'todo.count',
       'venue.id', 'venue.name', 'venue.location.address',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups',
       'venue.location.crossStreet', 'venue.location.state',
       'venue.venuePage.id', 'photo.tags', 'likes.summary',
       'venue.location.neighborhood'],
      dtype='object')

In [74]:
filterd_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
df_usertips_filtered = df_usertips.loc[:, filterd_columns]
df_usertips_filtered.sort_values('agreeCount', ascending = False)

,text,agreeCount,disagreeCount,id
6,"Saturday 8-10PM all you can drink beer, Prosecco and wine for something like SGD 30. Probably one of the better deals in town, superb view. Not the most popular terrace bar though...yet?",4,0,595b9bdb851de535fd11bf29
21,"Head to the roof top, there's a bar there. Not sure if hidden gem or not but very relaxed with view over China Town, Club Street and the Tooth Temple. Mojitos sucked though. (Level 4)",3,0,5661d9b2498ec523815ea3e1
26,Amazingly authentic tacos. Chef is Mexican. And if you didn't get it: try the tacos (al pastor). Michelada is also recommendable.,3,0,565f1770498eaaf0c0b4ed59
25,"Big, bigger, busier, Mustafa. Escape the crowded shop levels to the rooftop: there's an excellent Indian restaurant up there!",3,0,565f17f6498efbb91234857b
20,"15 months old. Try the surprise menu. For SGD 68, 4 courses excellent price/value. Pigeon with foie gras and rhubarb was superb as was the olive oil cake desert. Gourmets will concur.",2,0,566aa86b498e4018b9302650
16,Surprisingly authentic Greek cuisine with prices in a reasonable range. Great ambience with Greek music and the Santorini white washed walls and furniture. Try the Gyros and eggplant starter.,2,0,56d4f575498efff4429b6d71
23,"Don't know about the kebabs but the curries and Indian dishes we tried were great. Especially the southern Indian dishes, chicken tikka masala and butter chicken.",2,0,565f18d3498e1559915ab0d1
12,Go to Nespresso and have a free coffee. #PeasantLife,2,0,57596291498e8b297ea98104
13,Great new Japanese place. Roastbeef don special for lunch. Salmon avocado and California hand rolls were delicious. Good for a quick bite.,1,0,5745833c498edf63a709b0d7
19,Try the sliced fish soup.,1,0,566aa9e9498e746a9ec97553


In [85]:
tip_id = '595b9bdb851de535fd11bf29'

url = 'https://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION) # define URL

result_tip_id = requests.get(url).json()
result_tip_id['response']['tip']['venue']['name']
result_tip_id['response']['tip']['venue']['location']['address']

'Lv 43, Ocean Financial Centre'

### 4. Exploring venues around a given location

In [97]:
LAT = 48.151427
LNG = 11.572119
RADIUS = 1000

In [98]:
url_explore = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, LAT, LNG, VERSION, RADIUS, LIMIT)

In [99]:
result_explore = requests.get(url_explore).json()
result_explore

{'meta': {'code': 200, 'requestId': '61755336eaeed07dabadde77'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Maxvorstadt',
  'headerFullLocation': 'Maxvorstadt, Munich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 175,
  'suggestedBounds': {'ne': {'lat': 48.16042700900001,
    'lng': 11.585583758388431},
   'sw': {'lat': 48.14242699099999, 'lng': 11.55865424161157}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ccc5b967c2ff04decab987e',
       'name': 'Gegenüber',
       'location': {'address': 'Barer Str. 80',
        'lat': 48.152607170668084,
        'lng': 11.5742885117511

In [101]:
'there are {} venues arount the specified restaurant.'.format(len(result_explore['response']['groups'][0]['items']))

'there are 30 venues arount the specified restaurant.'

In [102]:
items = result_explore['response']['groups'][0]['items']

items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ccc5b967c2ff04decab987e',
  'name': 'Gegenüber',
  'location': {'address': 'Barer Str. 80',
   'lat': 48.152607170668084,
   'lng': 11.574288511751197,
   'labeledLatLngs': [{'label': 'display',
     'lat': 48.152607170668084,
     'lng': 11.574288511751197}],
   'distance': 207,
   'postalCode': '80799',
   'cc': 'DE',
   'city': 'München',
   'state': 'Bayern',
   'country': 'Deutschland',
   'formattedAddress': ['Barer Str. 80', '80799 München']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4ccc5b967c2ff04decab987e-0'}

In [103]:
df_explore = pd.json_normalize(items)
df_explore.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.neighborhood', 'venue.location.crossStreet',
       'venue.venuePage.id'],
      dtype='object')

In [105]:
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in df_explore.columns if col.startswith('venue.location')] + ['venue.id']
df_explore_filtered = df_explore.loc[:, filtered_columns]

df_explore_filtered['venue.categories'] = df_explore_filtered.apply(get_category_type, axis = 1)

df_explore_filtered.head()

,venue.name,venue.categories,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.neighborhood,venue.location.crossStreet,venue.id
0,Gegenüber,Café,Barer Str. 80,48.152607,11.574289,"[{'label': 'display', 'lat': 48.152607170668084, 'lng': 11.574288511751197}]",207,80799,DE,München,Bayern,Deutschland,"[Barer Str. 80, 80799 München]",NaN,NaN,4ccc5b967c2ff04decab987e
1,FOX Bar,Bar,Barer Str. 47,48.151574,11.573571,"[{'label': 'display', 'lat': 48.15157379179224, 'lng': 11.573571444161873}]",109,80799,DE,München,Bayern,Deutschland,"[Barer Str. 47, 80799 München]",Maxvorstadt,NaN,538f486d498e41ee060b9014
2,Boulangerie Dompierre,Bakery,Schellingstr. 34,48.150480,11.575199,"[{'label': 'display', 'lat': 48.15047968073459, 'lng': 11.575198535322926}]",251,80799,DE,München,Bayern,Deutschland,"[Schellingstr. 34, 80799 München]","Maxvorstadt, München",NaN,5335597a498e375511e7e5f9
3,PicNic,Modern European Restaurant,Barer Str. 48,48.149642,11.572370,"[{'label': 'display', 'lat': 48.149642082665466, 'lng': 11.572369824549304}]",199,80799,DE,München,Bayern,Deutschland,"[Barer Str. 48, 80799 München]",NaN,NaN,533d2a56498ebfd641d877f8
4,Bagels & Muffins,Bagel Shop,Barer Str. 72,48.151936,11.573845,"[{'label': 'display', 'lat': 48.1519358119813, 'lng': 11.573844777110887}]",140,80799,DE,München,Bayern,Deutschland,"[Barer Str. 72 (Blütenstr.), 80799 München]",NaN,Blütenstr.,4b9bf7abf964a5209d3b36e3


In [108]:
df_explore_filtered.columns = [col.split('.')[-1] for col in df_explore_filtered.columns]
df_explore_filtered['categories'].value_counts()

Café                          6
Italian Restaurant            2
Bakery                        2
Roof Deck                     1
Pizza Place                   1
German Restaurant             1
Restaurant                    1
Chinese Restaurant            1
Ice Cream Shop                1
Trattoria/Osteria             1
Bagel Shop                    1
Art Museum                    1
Boutique                      1
Kebab Restaurant              1
Tea Room                      1
Event Space                   1
Poke Place                    1
Bar                           1
Pastry Shop                   1
Pub                           1
Modern European Restaurant    1
Japanese Restaurant           1
Breakfast Spot                1
Name: categories, dtype: int64

In [111]:
explore_map = folium.Map([LAT, LNG], zoom_start = 16)

folium.CircleMarker([LAT, LNG],
                  radius = 20,
                  color = 'seagreen',
                  fill = True,
                  fill_color = 'seagreen',
                  fill_opacity = 0.5,
                  popup = 'Osteria Italiana').add_to(explore_map)

for lat, lng, name in zip(df_explore_filtered.lat, df_explore_filtered.lng, df_explore_filtered.name):
    folium.Marker([lat, lng],
                 popup = name,
                 icon = folium.Icon(
                     color = 'green')).add_to(explore_map)
explore_map

### 5. Exploring trending venues

In [129]:
LATITUDE = 20.00
LONGITUDE = 37.00
url_trending = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, LATITUDE, LONGITUDE, VERSION, RADIUS, LIMIT)
result_trending = requests.get(url_trending).json()
result_trending

{'meta': {'code': 200, 'requestId': '61755d2fd95d4b22eb57fc95'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': []}}

In [131]:
if len(result_trending['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment.'
else:
    trending_venues = result_trending['response']['venues']
    trending_venues_df = pd.json_normalize(trending_venues)
    
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_filtered = trending_venues_df.loc[:,columns_filtered]

    trending_venues_filtered.categories = trending_venues_filtered.apply(get_category_type, axis = 1)
                                

In [132]:
trending_venues_df

'No trending venues are available at the moment.'

In [134]:
if len(result_trending['response']['venues']) == 0:
    map_trending = 'Cannot generate visual since no trending venues are available at the moment.'

else:
    map_trending = folium.Map([LATITUDE, LONGITUDE], zoom_start = 16)
    folium.CircleMarker([LATITUDE, LONGITUDE],
                       color = 'seagreen',
                        radius = 20,
                       fill = True,
                       fill_color = 'seagreen', 
                       fill_opacity = 0.5,
                       popup = 'start point').add_to(map_trending)
    for lat, lng, name in zip(trending_venues_filtered['location.lat'], trending_venues_filtered['locaiton.lng'], trending_venues_filtered['name']):
        folium.Marker([lat, lng],
                     popup = name, 
                     icon = folium.Icon(color = 'green')).add_to(map_trending)

map_trending

'Cannot generate visual since no trending venues are available at the moment.'